# Loading and Plotting a DEM which is online on some kind of cloud storage service.


In [ ]:
## 1. Install the necessary libraries if they are not installed in your environment
#!pip install rasterio matplotlib

In [9]:
## 2. Load the libraries 
import rasterio # a package with functions for working with raster data and plotting them
from rasterio.plot import show 
from matplotlib import pyplot as plt # plotting and visualization library 
import numpy as np # a package for handling numeric arrays and matricies

In [ ]:
## 3. Load the URL using Rasterio and the path to the file
url = 'https://atur-karst.s3.us-east-2.amazonaws.com/MMRK1m.tif'

with rasterio.open(url) as src:
    dem = src.read(1)
    profile = src.profile

## 4. Create a basic function to create a hillshade from the .tif file

What the function does line by line:
```
x,y = np.gradient(dem)
```
'np.gradient(dem)' is a function that computes the gradiant of an array, when applied to a 2D array, it computes the partial derivatives along each access in this context, the DEM is a 2D array where each cell represents an elevation value. It returns two arrays, one for the gradient in the x direction, and one for the gradient in the y direction stored as variables 'x' and 'y'

why we need gradients? To create a hillshade we need to calculate both the slope and aspect of each pixel. The slope is calculated by combining the x and y gradients for each pixel. the aspect is the direction of the slope and is calculated using the ratio of the gradients. This determines the orientation of the slope relative to north.

```
slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
```
'np.arctan(np.sqrt(x*x + y*y))' calculates the inverse tangent of the gradient maginitude. the arctangent function returns the angle whose tangent is the given value, in this case it returns the angle of the slope in radians relative to the horizontal plane

'np.pi/2.' represents 90 degrees in radians, which is a vertical slope.

subtracting the arctangent value from 'np.pi/2.' converts the angle from a horizontal reference to a vertical value

Conceptually, this step translates the rate of change in elevation (the gradient) into the angle that repsents the steepness of the terrain

In [ ]:
## 4. Create a basic function to create a hillshade from the .tif file
def hillshade(dem, azimuth=315, angle_altitude=45):
    x, y = np.gradient(dem)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y)) # calculate slope
    aspect = np.arctan2(-x, y) # calcuate aspect
    azimuth_rad = azimuth * np.pi / 180. # set the azimuth or sun direction in degrees from north 
    altitude_rad = angle_altitude * np.pi / 180. # set the altitude of the sun in degrees above horizontal
    shaded = np.sin(altitude_rad) * np.sin(slope) + np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    return 255 * (shaded + 1) / 2

hillshade_dem = hillshade(dem)

In [ ]:
## 5. Plot the hillshade using matplot lib 

plt.figure(figsize=(10, 10)) # set the size of the blank canvas 10 x 10 inches
plt.imshow(hillshade_dem, cmap='terrain') # create a plot using the hillshade of the DEM 
plt.title('Hillshade') # create a title for the plot
plt.axis('off') # format the axes
plt.colorbar() # plot the color bar
plt.show() # display the plot 


In [ ]:
# demonstrate with #3DEP data 